In [ ]:
import os, sys
import sumolib
import libtraci as traci
import time
import pandas as pd
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, SubElement, ElementTree
import numpy as np
import shutil
import traci.constants
from multiprocessing import Pool
import gym
from gym import spaces
import random
from tensorforce.environments import Environment
from tensorforce.agents import Agent
from tensorforce.execution import Runner

In [ ]:
## Simulation environment
class TrafficSimEnv(Environment):
    def __init__(self):
        super().__init__()
        self.dtype = 'high'
        self.demand = np.ones(28, dtype='float')

    def states(self):
        return dict(type='float', shape=(28,))
    
    def actions(self):
        return dict(type='float', shape=(1,), min_value = 0.05, max_value= 0.059)
    
    def close(self):
        super().close()
        
    def reset(self):
        return np.ones(28, dtype='float')
    
    def execute(self, actions):
        next_state = self.demand
        terminal = False
        
        self.make_add_edge_xml()
        self.run_simulation()
        eval_speed, eval_volume = self.simulation_evaluation()
        reward = (100 - abs(eval_speed - 100)) * 0.08 + (100 - abs(eval_volume - 100)) * 0.02
        print("reward : {:.2f}".format(reward))
        print("demand : {}".format(self.demand))
        print("Speed_acc : {}".format(eval_speed) + " Volume_acc : {}".format(eval_volume))
        
        self.demand[random.randint(0,27)] -= round(actions[0],2)
        next_state = self.demand
        
        if reward >= 0.9:
            print('check')
            terminal = True
            
        return next_state, terminal, reward

    